In [80]:
import numpy as np
import pandas as pd
from datetime import datetime,time,timedelta
import pandas_bokeh


In [62]:
pandas_bokeh.output_notebook()

Loading BokehJS ...

## Carregar Dados

In [63]:
df = pd.read_excel('Data.xls',parse_dates=['Data Chamado'])

### Ver os tipos de dados do  Dataset

In [64]:
df.dtypes

Data Chamado          datetime64[ns]
Data Resposta         datetime64[ns]
Tempo Resposta (s)            object
Usuário                       object
Atendente                     object
Problema                      object
Status                        object
dtype: object

# Processo de ETL

![img](https://raw.githubusercontent.com/EloneSampaio/DataScience/master/Dashboard%20Bancario/ETL3.jpg)

## T de Transform

  - #### Nesta etapa transformamos nossos dados fazendo conversões de tipo de dados, limpeza de valores nulos, etc

### Conversões de tipo de dados

In [65]:
# conversão da data no formato internacional

df['Data Chamado'] = df['Data Chamado'].dt.strftime('%d/%m/%Y')
df['Data Resposta'] = df['Data Resposta'].dt.strftime('%d/%m/%Y')



## Cast dos campos


df['Data Chamado'] = df['Data Chamado'].astype('datetime64[ns]')
df['Data Resposta'] = df['Data Resposta'].astype('datetime64[ns]') 
df['Tempo Resposta (s)'] =  pd.to_datetime(df['Tempo Resposta (s)'],format='%H:%M:%S')

In [66]:
df.head()

,Data Chamado,Data Resposta,Tempo Resposta (s),Usuário,Atendente,Problema,Status
0,2019-01-01,2019-10-01,1900-01-01 00:01:10,Manuela Abreu Braga Martins,Camila Pereira,Alterar senha,Encerrado
1,2019-01-01,2019-03-01,1900-01-01 00:04:43,Rafael Viana Santos,Alon Pinheiro,Problema com login,Sem resposta
2,2019-01-01,2019-04-01,1900-01-01 00:04:08,Adelino Werneck,Diego Amorim,Dúvidas/Outros,Encerrado
3,2019-01-01,2019-01-01,1900-01-01 00:03:07,Flávio Jacques Gamboa Fernandez de Oliveira Netto,Diego Amorim,Conexão lenta,Encerrado
4,2019-02-01,2019-03-01,1900-01-01 00:04:19,Raphael Rodrigues,Heitor Prado,Conexão lenta,Encerrado


In [67]:
df.dtypes

Data Chamado          datetime64[ns]
Data Resposta         datetime64[ns]
Tempo Resposta (s)    datetime64[ns]
Usuário                       object
Atendente                     object
Problema                      object
Status                        object
dtype: object

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Data Chamado        1059 non-null   datetime64[ns]
 1   Data Resposta       1059 non-null   datetime64[ns]
 2   Tempo Resposta (s)  1059 non-null   datetime64[ns]
 3   Usuário             1059 non-null   object        
 4   Atendente           1059 non-null   object        
 5   Problema            1059 non-null   object        
 6   Status              1059 non-null   object        
dtypes: datetime64[ns](3), object(4)
memory usage: 58.0+ KB


##  Metricas principais
#### Insights dos dados

In [69]:
#Média do Tempo Resposta

df['Tempo Resposta (s)'].mean()

Timestamp('1900-01-01 00:05:18.103871488')

In [70]:
# Metricas principais

#Total de atendentes

TotalAtendentes=len(df['Atendente'].unique())
TotalLigacoes = len(df)

# % de chamados resolvidos
filtro1 = df['Status'].isin(['Encerrado'])
#número de ligações resolvidas
ligacoes_resolvidas = len (df[filtro1])
percentual_resolvidos = round((ligacoes_resolvidas/ TotalLigacoes) *100,1)


#Chamados por status

chamado_status = df.groupby(['Status'])\
                   .agg({'Status': 'count'})[ ['Status'] ]\
                   .apply(lambda x: round(100*(x/x.sum()),1) ) 


#Chamados por problema
chamado_por_problema = df.groupby(['Problema'])\
                         .agg({'Problema': [('qtd','count')]})
chamado_por_problema.columns = chamado_por_problema.columns.droplevel()
chamado_por_problema.reset_index(inplace=True)
chamado_por_problema.sort_values(by=['qtd'], ascending=False,inplace=True)



#Chamados por atendentes

chamado_por_atendente = df.groupby(['Atendente'])\
                        .agg({'Data Resposta':[('chamadas','count')], 'Tempo Resposta (s)':[('Temp Médio','mean')]})
chamado_por_atendente.columns= chamado_por_atendente.columns.droplevel()
chamado_por_atendente.reset_index(inplace=True)
chamado_por_atendente.sort_values(by=['chamadas','Temp Médio'], ascending=False, inplace=True)

#Retornar os top 5
chamado_por_atendente = chamado_por_atendente.iloc[0:5,:]



#chamados por casos abertos e fechados ao longo do tempo

filtro2 = df['Status'].isin(['Encerrado','Sem resposta'])
chamados_aberto_fechado = df[filtro2]

chamados_aberto_fechado['contagem'] = chamados_aberto_fechado.groupby(['Status','Data Resposta']).cumcount()+1
chamados_aberto_fechado= chamados_aberto_fechado[ ['Status','Data Resposta','contagem' ] ].copy()
 

    
## Tempo médio por período

tempo_medio_por_data = df.groupby(['Data Resposta'])\
                         .agg({'Tempo Resposta (s)':[('tempo','mean')]})
tempo_medio_por_data.columns = tempo_medio_por_data.columns.droplevel()
tempo_medio_por_data.reset_index(inplace=True)

tempo_medio_por_data ['Tempo'] =pd.to_datetime( tempo_medio_por_data ['tempo']).dt.time
tempo_medio_por_data.drop(columns=['tempo'],inplace=True)

<ipython-input-70-69bdf8f5976e>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chamados_aberto_fechado['contagem'] = chamados_aberto_fechado.groupby(['Status','Data Resposta']).cumcount()+1


In [71]:
tempo_medio_por_data.head()

,Data Resposta,Tempo
0,2019-01-01,00:03:07
1,2019-01-02,00:07:23
2,2019-01-03,00:06:59.142857
3,2019-01-05,00:02:46.500000
4,2019-01-06,00:05:27


In [72]:
chamado_por_atendente.head()

,Atendente,chamadas,Temp Médio
3,Diego Amorim,305,1900-01-01 00:04:56.295081728
5,João Paulo,289,1900-01-01 00:05:42.505190400
4,Heitor Prado,146,1900-01-01 00:05:44.753424896
0,Alon Pinheiro,115,1900-01-01 00:05:21.226087168
2,Carol Espinosa,109,1900-01-01 00:06:23.449541120


In [73]:
chamados_aberto_fechado.head()

,Status,Data Resposta,contagem
0,Encerrado,2019-10-01,1
1,Sem resposta,2019-03-01,1
2,Encerrado,2019-04-01,1
3,Encerrado,2019-01-01,1
4,Encerrado,2019-03-01,1


# Visualização

In [74]:
chamados_aberto_fechado.plot_bokeh(kind='bar', stacked=True,show_figure=False)

Figure(id='2057', ...)

In [75]:
chamado_por_atendente.plot_bokeh.pie(y='chamadas',x='Atendente')

__x__values_original


Figure(id='2153', ...)

In [76]:
tempo_medio_por_data['Month'] = tempo_medio_por_data['Data Resposta'].apply(lambda x: x.month) 
tempo_medio_por_data['Year'] = tempo_medio_por_data['Data Resposta'].apply(lambda x: x.year)

In [77]:
tempo_medio_por_data.tail()

,Data Resposta,Tempo,Month,Year
341,2019-12-27,00:07:30.500000,12,2019
342,2019-12-28,00:03:15.500000,12,2019
343,2019-12-29,00:08:05.333333,12,2019
344,2019-12-30,00:03:35,12,2019
345,2019-12-31,00:01:41,12,2019


In [82]:
chamado_por_atendente.head()

,Atendente,chamadas,Temp Médio
3,Diego Amorim,305,1900-01-01 00:04:56.295081728
5,João Paulo,289,1900-01-01 00:05:42.505190400
4,Heitor Prado,146,1900-01-01 00:05:44.753424896
0,Alon Pinheiro,115,1900-01-01 00:05:21.226087168
2,Carol Espinosa,109,1900-01-01 00:06:23.449541120


## L de Load

  - #### Nesta etapa exportaremos nossos dados no formato excel para carregar no Data WareHouse

In [11]:
df.to_excel('chamados.xlsx',index=False)